In [1]:
from __future__ import print_function

import glob
from itertools import chain
import os
import random
import cv2  # Assuming you have OpenCV installed
import torchvision
from torchvision.datasets import ImageFolder
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import cv2
from linformer import Linformer
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
import itertools
import math
from vit_pytorch.efficient import ViT

In [2]:
# Required Parameters
dataset = "C:\\Study\\OJT\\Dataset\\HDMB"         # Dataset path
dataset2 = "C:\\Study\\OJT\\Dataset_Extraction\\HDMB\\Full"               # Dataset2 path
train_path = "C:\\Study\\OJT\\Dataset_Extraction\\HDMB\\HDMBTrain"            # Training path
test_path = "C:\\Study\\OJT\\Dataset_Extraction\\HDMB\\HDMBTest"            # Testing path
no_of_frames = 10000                     # Total number of frames to be extracted
categories = os.listdir(dataset)        # Name of each class/category

In [3]:
# Creating dataset directory
try:
    os.mkdir(dataset2)
    print("Folder {} created...".format(dataset2))
except:
    print("A folder {} already exists...".format(dataset2))

A folder C:\Study\OJT\Dataset_Extraction\HDMB\Full already exists...


In [4]:
# Creating training_set directory
try:
    os.mkdir(train_path)
    print("Folder {} created...".format(train_path))
except:
    print("A folder {} already exists...".format(train_path))

A folder C:\Study\OJT\Dataset_Extraction\HDMB\HDMBTrain already exists...


In [5]:
# Creating testing_set directory
try:
    os.mkdir(test_path)
    print("Folder {} created...".format(test_path))
except:
    print("A folder {} already exists...".format(test_path))

A folder C:\Study\OJT\Dataset_Extraction\HDMB\HDMBTest already exists...


In [6]:
# Creating same directories for dataset2/ that are already present in the dataset directory
for category in categories:
    try:
        os.mkdir(os.path.join(dataset2, category))
        print("Folder {} created...".format(category))
    except:
        print("A folder already exists, named {}...".format(category, dataset))

Folder brush_hair created...
Folder cartwheel created...
Folder catch created...
Folder chew created...
Folder clap created...
Folder climb created...
Folder climb_stairs created...
Folder dive created...
Folder draw_sword created...
Folder dribble created...
Folder drink created...
Folder eat created...
Folder fall_floor created...
Folder fencing created...
Folder flic_flac created...
Folder golf created...
Folder handstand created...
Folder hit created...
Folder hug created...
Folder jump created...
Folder kick created...
Folder kick_ball created...
Folder kiss created...
Folder laugh created...
Folder pick created...
Folder pour created...
Folder pullup created...
Folder punch created...
Folder push created...
Folder pushup created...
Folder ride_bike created...
Folder ride_horse created...
Folder run created...
Folder shake_hands created...
Folder shoot_ball created...
Folder shoot_bow created...
Folder shoot_gun created...
Folder sit created...
Folder situp created...
Folder smile

In [7]:
for category in categories:
    try:
        os.mkdir(os.path.join(train_path, category))
        print("Folder {} created...".format(category))
    except:
        print("A folder already exists, named {}...".format(category, train_path))


Folder brush_hair created...
Folder cartwheel created...
Folder catch created...
Folder chew created...
Folder clap created...
Folder climb created...
Folder climb_stairs created...
Folder dive created...
Folder draw_sword created...
Folder dribble created...
Folder drink created...
Folder eat created...
Folder fall_floor created...
Folder fencing created...
Folder flic_flac created...
Folder golf created...
Folder handstand created...
Folder hit created...
Folder hug created...
Folder jump created...
Folder kick created...
Folder kick_ball created...
Folder kiss created...
Folder laugh created...
Folder pick created...
Folder pour created...
Folder pullup created...
Folder punch created...
Folder push created...
Folder pushup created...
Folder ride_bike created...
Folder ride_horse created...
Folder run created...
Folder shake_hands created...
Folder shoot_ball created...
Folder shoot_bow created...
Folder shoot_gun created...
Folder sit created...
Folder situp created...
Folder smile

In [8]:
# Creating same directories for testing_set/ that are already present in the dataset directory
for category in categories:
    try:
        os.mkdir(os.path.join(test_path, category))
        print("Folder {} created...".format(category))
    except:
        print("A folder already exists, named {}...".format(category, test_path))


Folder brush_hair created...
Folder cartwheel created...
Folder catch created...
Folder chew created...
Folder clap created...
Folder climb created...
Folder climb_stairs created...
Folder dive created...
Folder draw_sword created...
Folder dribble created...
Folder drink created...
Folder eat created...
Folder fall_floor created...
Folder fencing created...
Folder flic_flac created...
Folder golf created...
Folder handstand created...
Folder hit created...
Folder hug created...
Folder jump created...
Folder kick created...
Folder kick_ball created...
Folder kiss created...
Folder laugh created...
Folder pick created...
Folder pour created...
Folder pullup created...
Folder punch created...
Folder push created...
Folder pushup created...
Folder ride_bike created...
Folder ride_horse created...
Folder run created...
Folder shake_hands created...
Folder shoot_ball created...
Folder shoot_bow created...
Folder shoot_gun created...
Folder sit created...
Folder situp created...
Folder smile

In [9]:
# Combining multiple videos into single video file
for category in tqdm(categories):
    videofiles = glob.glob(os.path.join(dataset, category, "**/*.avi"), recursive=True)
    if videofiles:
        cap = cv2.VideoCapture(videofiles[0])
    else:
        print("No video files found in {}/{}".format(dataset, category))
    video_index = 0
    cap = cv2.VideoCapture(videofiles[0])    
    fourcc = cv2.VideoWriter_fourcc(*"XVID")
    # fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    out = cv2.VideoWriter("{}/{}/{}.avi".format(dataset2, category, category), fourcc, 25, (320, 240))
    while(cap.isOpened()):
        ret, frame = cap.read()
        if frame is None:
            video_index += 1
            if video_index >= len(videofiles):
                break
            else:
                cap = cv2.VideoCapture(videofiles[ video_index ])
                ret, frame = cap.read()
                out.write(frame)
        else:
            out.write(frame)
    cap.release()
    out.release()
    

  0%|          | 0/51 [00:00<?, ?it/s]

In [10]:
# Saving total no. of frames of each classes/categories into an array
total_frames = []
for category in tqdm(categories):
    cap = cv2.VideoCapture(dataset2 + "/" + category + "/" + category + ".avi")
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    total_frames.append(length)
    cap.release()
    
print(total_frames)

  0%|          | 0/51 [00:00<?, ?it/s]

[23568, 6323, 2496, 8390, 7386, 15656, 4505, 6458, 6144, 6521, 1593, 942, 440, 5092, 6473, 6820, 8112, 3042, 5188, 1356, 1571, 4060, 11018, 20026, 4706, 16864, 6888, 3609, 8165, 8087, 7433, 12346, 4307, 5974, 8254, 19333, 7753, 1771, 7235, 5736, 9619, 7374, 2944, 10541, 4943, 7535, 346, 7727, 2856, 11534, 3984]


In [11]:
# Extracting one frame per five frame from the Videos
for category in tqdm(categories):
    count = 0    
    a = glob.glob(dataset2 + '/' + category + '/' + category +'.avi')
    for i in range(len(a)):
        cap = cv2.VideoCapture(a[i])
        frameRate = cap.get(5)
        while(cap.isOpened()):
            frameId = cap.get(1)
            ret, frame = cap.read()
            if (ret != True):
                break
            if (frameId % math.floor(frameRate) == 0):
                cv2.imwrite(train_path + '/' + category + '/{}_{}.jpg'.format(category, count), frame)
                count += 1
        cap.release()

  0%|          | 0/51 [00:00<?, ?it/s]

In [13]:
# Moving 150 random images from training_set into testing_set
for category in tqdm(categories):
    sub_file = [file for file in glob.glob(train_path +'/'+ category +'/'+ "*.jpg")]
    test_files = random.sample(sub_file, 10)
    for test_file in test_files:
        img = cv2.imread(test_file)
        os.remove(test_file)
        test_filename = os.path.basename(test_file)
        cv2.imwrite(test_path +'/' + category + '/' + test_filename , img)
        

  0%|          | 0/51 [00:00<?, ?it/s]

In [14]:
# Training settings
batch_size = 64
epochs = 20
lr = 0.001
gamma = 0.7
seed = 42

In [15]:
# Define the transformation for loading and preprocessing images
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),  # You can adjust the crop size as needed
    transforms.ToTensor(),
])

In [16]:
# Create a custom dataset for the entire training set
full_dataset = ImageFolder(train_path, transform=transform)

In [17]:
# Split the dataset into training and validation sets
train_size = int(0.8 * len(full_dataset))  # 80% for training
val_size = len(full_dataset) - train_size  # 20% for validation
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

In [18]:
# Create custom datasets for training and test
train_dataset = ImageFolder(train_path, transform=transform)
test_dataset = ImageFolder(test_path, transform=transform)

In [19]:
# Create data loaders for training and test sets
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [20]:
efficient_transformer = Linformer(
    dim=128,
    seq_len=49+1,  # 7x7 patches + 1 cls-token
    depth=12,
    heads=8,
    k=64
)

In [21]:
model = ViT(
    dim=128,
    image_size=224,
    patch_size=32,
    num_classes=51,
    transformer=efficient_transformer,
    channels=3,
)

In [22]:
# loss function
criterion = nn.CrossEntropyLoss()
# optimizer
optimizer = optim.Adam(model.parameters(), lr=lr)
# scheduler
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

In [24]:
# Training loop
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        if batch_idx % 10 == 9:  # Print every 10 batches
            print(f"Epoch {epoch+1}, Batch {batch_idx+1}/{len(train_loader)}, Loss: {running_loss / 10:.4f}, Training Accuracy: {100 * correct / total:.2f}%")
            running_loss = 0.0

    scheduler.step()

    # Validation accuracy calculation
    model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for val_inputs, val_labels in val_loader:
            val_outputs = model(val_inputs)
            _, val_predicted = torch.max(val_outputs.data, 1)
            val_total += val_labels.size(0)
            val_correct += (val_predicted == val_labels).sum().item()

    val_accuracy = 100 * val_correct / val_total
    print(f"Epoch {epoch+1}, Training Accuracy: {100 * correct / total:.2f}%, Validation Accuracy: {val_accuracy:.2f}%")

# Evaluation on the test set
model.eval()
test_correct = 0
test_total = 0

with torch.no_grad():
    for test_inputs, test_labels in test_loader:
        test_outputs = model(test_inputs)
        _, test_predicted = torch.max(test_outputs.data, 1)
        test_total += test_labels.size(0)
        test_correct += (test_predicted == test_labels).sum().item()

print(f"Test Accuracy: {100 * test_correct / test_total:.2f}%")
torch.save(model.state_dict(), 'ViT_HDMB_pth')

Epoch 1, Batch 10/210, Loss: 3.8115, Training Accuracy: 8.28%
Epoch 1, Batch 20/210, Loss: 3.7255, Training Accuracy: 7.81%
Epoch 1, Batch 30/210, Loss: 3.6287, Training Accuracy: 8.18%
Epoch 1, Batch 40/210, Loss: 3.5153, Training Accuracy: 8.98%
Epoch 1, Batch 50/210, Loss: 3.4471, Training Accuracy: 9.88%
Epoch 1, Batch 60/210, Loss: 3.2512, Training Accuracy: 11.02%
Epoch 1, Batch 70/210, Loss: 3.2306, Training Accuracy: 12.12%
Epoch 1, Batch 80/210, Loss: 3.0312, Training Accuracy: 13.38%
Epoch 1, Batch 90/210, Loss: 3.0305, Training Accuracy: 14.46%
Epoch 1, Batch 100/210, Loss: 2.8927, Training Accuracy: 15.73%
Epoch 1, Batch 110/210, Loss: 2.8245, Training Accuracy: 16.72%
Epoch 1, Batch 120/210, Loss: 2.7538, Training Accuracy: 17.90%
Epoch 1, Batch 130/210, Loss: 2.7681, Training Accuracy: 18.85%
Epoch 1, Batch 140/210, Loss: 2.6130, Training Accuracy: 20.15%
Epoch 1, Batch 150/210, Loss: 2.5598, Training Accuracy: 21.26%
Epoch 1, Batch 160/210, Loss: 2.4632, Training Accurac